<a href="https://colab.research.google.com/github/jlee1998/Advanced-Project-in-Mathematics/blob/main/Kernel_Flows_for_Irregular_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import autograd.numpy as np
import math
#%% Kernel operations

# Returns the norm of the pairwise difference
def norm_matrix(matrix_1, matrix_2):
    norm_square_1 = np.sum(np.square(matrix_1), axis = 1)
    norm_square_1 = np.reshape(norm_square_1, (-1,1))
    
    norm_square_2 = np.sum(np.square(matrix_2), axis = 1)
    norm_square_2 = np.reshape(norm_square_2, (-1,1))
    
    d1=matrix_1.shape
    d2=matrix_2.shape
#    print(d1)
#    print(d2)
    if d1[1]!=d2[1]:
        matrix_1=np.transpose(matrix_1)
    
    inner_matrix = np.matmul(matrix_1, np.transpose(matrix_2))
    
    norm_diff = -2 * inner_matrix + norm_square_1 + np.transpose(norm_square_2)
#    print(norm_diff.shape)
    
    return norm_diff

# Returns the pairwise inner product
def inner_matrix(matrix_1, matrix_2):
    d1=matrix_1.shape
    d2=matrix_2.shape
    # print(d1)
    # print(d2)
    if d1[1]!=d2[1]:
        matrix_1=np.transpose(matrix_1)
    return np.matmul(matrix_1, np.transpose(matrix_2))


if __name__ == '__main__':
    print('This is the matrix operations file')

This is the matrix operations file


In [2]:
""" In this section we define various kernels. Warning, not all of them work 
at the moment, the most reliable one is the RBF kernel. Note that currently the 
laplacian kernel does not work"""
        

# Define the RBF Kernel. Takes an array of parameters, returns a value
def kernel_RBF(matrix_1, matrix_2, parameters):
    matrix = norm_matrix(matrix_1, matrix_2)
    sigma = parameters[0]
    K =  np.exp(-matrix/(2*1.0**2))
    
    return K


# do not use right now
def kernel_laplacian(matrix_1, matrix_2, parameters):
    gamma = parameters[0]
    matrix = norm_matrix(matrix_1, matrix_2)
    K =  np.exp(-matrix * gamma)
    return K

def kernel_sigmoid(matrix_1, matrix_2, parameters):
    alpha = parameters[0]
    beta = parameters[1]
    matrix = inner_matrix(matrix_1, matrix_2)
    K = np.tanh(alpha *matrix + beta)
    return K

def kernel_rational_quadratic(matrix_1, matrix_2, parameters):
    alpha = parameters[0]
    beta = parameters[1]
    epsilon = 0.0001
    matrix = norm_matrix(matrix_1, matrix_2)
    return (beta**2 + matrix)**(-(alpha+ epsilon))

def kernel_inverse_power_alpha(matrix_1, matrix_2, parameters):
    alpha = parameters[0]
    beta = 1.0
    epsilon = 0.0001
    matrix = norm_matrix(matrix_1, matrix_2)
    return (beta**2 + matrix)**(-(alpha+ epsilon))

def kernel_inverse_multiquad(matrix_1, matrix_2, parameters):
    beta = parameters[0]
    gamma = parameters[1]
    matrix = norm_matrix(matrix_1, matrix_2)
    return (beta**2 + gamma*matrix)**(-1/2)

def kernel_cauchy(matrix_1, matrix_2, parameters):
    sigma = parameters[0]
    matrix = norm_matrix(matrix_1, matrix_2)
    return 1/(1 + matrix/sigma**2)

def kernel_quad(matrix_1, matrix_2, parameters):
    c = parameters[0]
    matrix = inner_matrix(matrix_1, matrix_2)
    K = (matrix+c) ** 2
    return K 

def kernel_poly(matrix_1, matrix_2, parameters):
    a = parameters[0]
    b = parameters[1]
    d = parameters[2]
    matrix = inner_matrix(matrix_1, matrix_2)
    K = (a * matrix + b) ** d
    return K 


def kernel_gaussian_linear(matrix_1, matrix_2, parameters):
    K = 0
    matrix = norm_matrix(matrix_1, matrix_2)
    for i in range(parameters.shape[1]):
        # print("beta", parameters[1, i])
        # print("sigma", parameters[0, i])
        K = K + parameters[1, i]**2*np.exp(-matrix / (2* parameters[0, i]**2))
    return K



def kernel_anl(matrix_1, matrix_2, parameters):
    matrix = norm_matrix(matrix_1, matrix_2)
    sigma = parameters[0]
    K =  np.exp(-matrix/ (2* sigma**2))
    K=K*(parameters[1])**2
    
    c = (parameters[2])**2
    matrix = inner_matrix(matrix_1, matrix_2)
    K = K+ (parameters[3])**2 *(matrix+c) ** 2
    
    beta = parameters[4]
    gamma = (parameters[5])**2
    matrix = norm_matrix(matrix_1, matrix_2)
    K=K+ (parameters[6])**2 *(beta**2 + gamma*matrix)**(-1/2)
    
    alpha = parameters[7]
    beta = parameters[8]
    matrix = norm_matrix(matrix_1, matrix_2)
    K=K+ (parameters[9])**2 *(beta**2 + matrix)**(-alpha)
    
    return K

def kernel_anl2(matrix_1, matrix_2, parameters):
    i=0
    
    matrix = norm_matrix(matrix_1, matrix_2)
    sigma = parameters[i+0]
    K =  np.exp(-matrix/ (2* sigma**2))
    K=K*(parameters[i+1])**2
    i=i+2
    
    
    c = (parameters[i])**2
    imatrix = inner_matrix(matrix_1, matrix_2)
    K = K+ (parameters[i+1])**2 *(imatrix+c) ** 2
    i=i+2
    
    beta = parameters[i]
    gamma = (parameters[i+1])**2
    K=K+ (parameters[i+2])**2 *(beta**2 + gamma*matrix)**(-1/2)
    i=i+3
    
    alpha = parameters[i]
    beta = parameters[i+1]
    K=K+ (parameters[i+2])**2 *(beta**2 + matrix)**(-alpha)
    i=i+3
    
    return K


def kernel_anl3(matrix_1, matrix_2, parameters):
    i=0
    
    matrix = norm_matrix(matrix_1, matrix_2)
    sigma = parameters[i+0]
    K =  np.exp(-matrix/ (2* sigma**2))
    K=K*(parameters[i+1])**2
    i=i+2
    
    
    c = (parameters[i])**2
    imatrix = inner_matrix(matrix_1, matrix_2)
    K = K+ (parameters[i+1])**2 *(imatrix+c) ** 2
    i=i+2
    
    beta = parameters[i]
    gamma = (parameters[i+1])**2
    K=K+ (parameters[i+2])**2 *(beta**2 + gamma*matrix)**(-1/2)
    i=i+3
    
    alpha = parameters[i]
    beta = parameters[i+1]
    K=K+ (parameters[i+2])**2 *(beta**2 + matrix)**(-alpha)
    i=i+3
    
    sigma = parameters[i]
    K=K+ (parameters[i+1])**2 * 1/(1 + matrix/sigma**2)
    i=i+2
    
    alpha_0 = parameters[i]
    sigma_0 = parameters[i+1]
    alpha_1 = parameters[i+2]
    sigma_1 = parameters[i+3]
    K =  K+ (parameters[i+4])**2 *alpha_0*np.maximum(0, 1-matrix/(sigma_0))+ alpha_1 * np.exp(-matrix/ (2* sigma_1**2))
    i=i+5
    
    p = parameters[i]
    l = parameters[i+1]
    sigma = parameters[i+2]
    K =K+ (parameters[i+3])**2 * np.exp(-np.sin(matrix*np.pi/p)**2/l**2)*np.exp(-matrix/sigma**2)
    i=i+4
    
    p = parameters[i]
    l = parameters[i+1]
    K = K+ (parameters[i+2])**2 *np.exp(-np.sin(matrix*np.pi/p)/l**2)
    i=i+3
    

    return K



"""A dictionnary containing the different kernels. If you wish to build a custom 
 kernel, add the function to the dictionnary.
"""
kernels_dic = {"RBF" : kernel_RBF,"poly": kernel_poly, "Laplacian": kernel_laplacian, 
               "sigmoid": kernel_sigmoid, "rational quadratic": kernel_rational_quadratic,
               "inverse_multiquad": kernel_inverse_multiquad, "quadratic" : kernel_quad,
               "poly": kernel_poly, "inverse_power_alpha": kernel_inverse_power_alpha,
               "gaussian multi": kernel_gaussian_linear, "anl": kernel_anl, "anl2": kernel_anl2,
               "anl3": kernel_anl3}


if __name__ == '__main__':
    print('This is the kernel file')

This is the kernel file


In [ ]:
!git clone https://github.com/KalmanNet/KalmanNet_TSP

Cloning into 'KalmanNet_TSP'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 50 (delta 13), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (50/50), done.


In [ ]:
def Henon(T, dt, N_sims,a,b):
    N_t  = int(T//dt)
    sims = np.zeros((N_sims, N_t, 2))
    for i in range(1,N_t):
      sims[:, i] = np.array([1-a*sims[:,i-1,0]**2+sims[:,i-1,1],b*sims[:,i-1,0]]).T
    return sims.astype(np.float32)

In [ ]:
# generate dataset
Data = Henon(T=10, dt=0.005,N_sims=1,a=1.4,b=0.3)[0]
#train test split
train_data = Data[:600,:]
test_data = Data[600:,:].T
#times
Times = np.linspace(1,2000-1,2000-1)
#take random subset
indices_random = np.sort(np.random.choice(600,360,replace=False))
#take train points corresponding to subset indices
train_times = Times[indices_random]
train_data = train_data[indices_random,:].T


In [ ]:
np.random.seed(1)
from autograd import value_and_grad 
import math
import matplotlib.pyplot as plt
def sample_selection(data, size):
  indices = np.arange(data.shape[0])
  sample_indices = np.sort(np.random.choice(indices, size, replace= False)) 
  return sample_indices

def batch_creation(data, batch_size, sample_proportion = 0.5):
  if batch_size == False:
    data_batch = data
    batch_indices = np.arange(data.shape[0])
  elif 0 < batch_size <= 1:
    batch_size = int(data.shape[0] * batch_size)
    batch_indices = sample_selection(data, batch_size)
    data_batch = data[batch_indices]
  else:
    batch_indices = sample_selection(data, batch_size)
    data_batch = data[batch_indices]
        

    # Sample from the mini-batch
  sample_size = math.ceil(data_batch.shape[0]*sample_proportion)
  sample_indices = sample_selection(data_batch, sample_size)
    
  return sample_indices, batch_indices


# Generate a prediction
def kernel_regression(X_train, X_test, Y_train, param, kernel_keyword = "RBF", regu_lambda = 0.000001):
    kernel = kernels_dic[kernel_keyword]
    k_matrix = kernel(X_train, X_train, param)
    k_matrix += regu_lambda * np.identity(k_matrix.shape[0])
    t_matrix = kernel(X_test, X_train, param) 
    prediction = np.matmul(t_matrix, np.matmul(np.linalg.inv(k_matrix), Y_train)) 
    return prediction

# redicttimeseries

def kernel_extrapolate(X_train, X_test, Y_train, param, nsteps=1, kernel_keyword = "RBF", regu_lambda = 0.000001):
    kernel = kernels_dic[kernel_keyword]
    k_matrix = kernel(X_train, X_train, param)
    k_matrix += regu_lambda * np.identity(k_matrix.shape[0])
    A=np.matmul(np.linalg.inv(k_matrix), Y_train)
    arr = np.array([])
    
    X_test0=X_test
    isteps=int(nsteps/(X_test.shape[1]))+1
    for i in range(isteps):
        X_test1=X_test0
        t_matrix = kernel(X_test1, X_train, param) 
        prediction = np.matmul(t_matrix, A) 
        X_test0=prediction
        arr = np.append(arr, np.array(prediction[0,:]))
    arr=arr[0:nsteps]
    return arr


def replace_nan(array):
    for i in range(array.shape[0]):
        if math.isnan(array[i]) == True:
            print("Found nan value, replacing by 0")
            array[i] = 0
    return array

def sample_size_linear(iterations, range_tuple):
    
    return np.linspace(range_tuple[0], range_tuple[1], num = iterations)[::-1]
            
#%% Rho function

# The pi or selection matrix
def pi_matrix(sample_indices, dimension):
    pi = np.zeros(dimension)
    
    for i in range(dimension[0]):
        pi[i][sample_indices[i]] = 1
    
    return pi


def rho(parameters, matrix_data, Y_data, sample_indices,  kernel_keyword= "RBF", regu_lambda = 0.000001):
    kernel = kernels_dic[kernel_keyword]
    
    kernel_matrix = kernel(matrix_data, matrix_data, parameters)
#    print(kernel_matrix.shape)
    pi = pi_matrix(sample_indices, (sample_indices.shape[0], matrix_data.shape[0]))   
#    print(pi.shape)
    
    sample_matrix = np.matmul(pi, np.matmul(kernel_matrix, np.transpose(pi)))
#    print(sample_matrix.shape)
    
    Y_sample = Y_data[sample_indices]
#    print(Y_sample.shape)
    
    lambda_term = regu_lambda
    inverse_data = np.linalg.inv(kernel_matrix + lambda_term * np.identity(kernel_matrix.shape[0]))
    inverse_sample = np.linalg.inv(sample_matrix + lambda_term * np.identity(sample_matrix.shape[0]))
#    print(inverse_sample.shape)
#    B=np.matmul(inverse_sample, Y_sample)
#    print(B.shape)
    
    top = np.tensordot(Y_sample, np.matmul(inverse_sample, Y_sample))
    
   
    bottom = np.tensordot(Y_data, np.matmul(inverse_data, Y_data))
    
    #print(1-top/bottom)
    return 1 - top/bottom

def l2(parameters, matrix_data, Y, batch_indices, sample_indices, kernel_keyword = "RBF"):
    X_sample = matrix_data[sample_indices]
    Y_sample = Y[sample_indices]
    
    not_sample = [x for x in batch_indices not in sample_indices]
    X_not_sample = matrix_data[not_sample]
    Y_not_sample = Y[not_sample]
    prediction = kernel_regression(X_sample, X_not_sample, Y_sample, kernel_keyword)
    
    return np.dot(Y_not_sample - prediction, Y_not_sample- prediction)

#%% Grad functions

""" We define the gradient calculator function.Like rho, the gradient 
calculator function accesses the gradfunctions via a keyword"""

# Gradient calculator function. Returns an array
def grad_kernel(parameters, X_data, Y_data, sample_indices, kernel_keyword= "RBF", regu_lambda = 0.000001):
    grad_K = value_and_grad(rho)
    rho_value, gradient = grad_K(parameters, X_data, Y_data, sample_indices, kernel_keyword, regu_lambda = regu_lambda)
    return rho_value, gradient

#%% The class version of KF
class KernelFlowsP():
    
    def __init__(self, kernel_keyword, parameters):
        self.kernel_keyword = kernel_keyword
        self.parameters = np.copy(parameters)
        
        # Lists that keep track of the history of the algorithm
        self.rho_values = []
        self.grad_hist = []
        self.para_hist = []
        
        self.LR = 0.1
        self.beta = 0.9
        self.regu_lambda = 0.0001
    
    def get_hist(self):
        return self.param_hist, self.gradients, self.rho_values
        
    
    def save_model(self):
        np.save("param_hist", self.param_hist)
        np.save("gradients", self.gradients)
        np.save("rho_values", self.rho_values)
        
    def get_parameters(self):
        return self.parameters
    
    def set_LR(self, value):
        self.LR = value
        
    def set_beta(self, value):
        self.beta = value
    def set_train(self, train):
        self.train = train
        
    
    def fit(self, X, Y, iterations, batch_size = False, optimizer = "SGD", 
            learning_rate = 0.1, beta = 0.9, show_it = 100, regu_lambda = 0.000001, 
            adaptive_size = False, adaptive_range = (), proportion = 0.5, reduction_constant = 0.0):            

        self.set_LR(learning_rate)
        self.set_beta(beta)
        self.regu_lambda = regu_lambda
        
        self.X_train = np.copy(X)
        self.Y_train = np.copy(Y)
        momentum = np.zeros(self.parameters.shape, dtype = "float")
        
        # This is used for the adaptive sample decay
        rho_100 = []
        adaptive_mean = 0
        adaptive_counter = 0
        
        if adaptive_size == False or adaptive_size == "Dynamic":
            sample_size = proportion
        elif adaptive_size == "Linear":
            sample_size_array = sample_size_linear(iterations, adaptive_range) 
        else:
            print("Sample size not recognized")
            
        for i in range(iterations-1):
            if i % show_it == 0:
                print("parameters ", self.parameters)
            
            if adaptive_size == "Linear":
                sample_size = sample_size_array[i]
                
            elif adaptive_size == "Dynamic" and adaptive_counter == 100:
                if adaptive_mean != 0:
                    change = np.mean(rho_100) - adaptive_mean 
                else:
                    change = 0
                adaptive_mean = np.mean(rho_100)
                rho_100 = []
                sample_size += change - reduction_constant
                adaptive_counter= 0
                
            # Create a batch and a sample
            sample_indices, batch_indices = batch_creation(X, batch_size, sample_proportion = sample_size)
            X_data = X[batch_indices]
            Y_data = Y[batch_indices]
            

                
            # Changes parameters according to SGD rules
            if optimizer == "SGD":
                rho, grad_mu = grad_kernel(self.parameters, X_data, Y_data, 
                                           sample_indices, self.kernel_keyword, regu_lambda = regu_lambda)
                #print(grad_mu)
                if  rho > 1 or rho < 0:
                    print("Warning, rho outside [0,1]: ", rho)
                else:
                    self.parameters -= learning_rate * grad_mu 
                    #* (Times[i]-train_times[i])
                    
            
            # Changes parameters according to Nesterov Momentum rules     
            elif optimizer == "Nesterov":
                rho, grad_mu = grad_kernel(self.parameters - learning_rate * beta * momentum, 
                                               X_data, Y_data, sample_indices, self.kernel_keyword, regu_lambda = regu_lambda)
                if  rho > 1 or rho < 0:
                    print("Warning, rho outside [0,1]: ", rho)
                else:
                    momentum = beta * momentum + grad_mu
                    self.parameters -= learning_rate * momentum * (train_times[i]-train_times[i-1])
                
            else:
                print("Error optimizer, name not recognized")
            
            # Update history 
            self.para_hist.append(np.copy(self.parameters))
            self.rho_values.append(rho)
            self.grad_hist.append(np.copy(grad_mu))
            
            rho_100.append(rho)
            adaptive_counter +=1
                
            
        # Convert all the lists to np arrays
        self.para_hist = np.array(self.para_hist) 
        self.rho_values = np.array(self.rho_values)
        self.grad_hist = np.array(self.grad_hist)
                
        return self.parameters
    
    def predict(self,test, regu_lambda = 0.0000001):
         
        X_train = self.X_train
        Y_train = self.Y_train
        prediction = kernel_regression(X_train, test, Y_train, self.parameters, self.kernel_keyword, regu_lambda = regu_lambda) 

        return prediction

    def extrapolate(self,test, nsteps=1,regu_lambda = 0.000001):
         
        X_train = self.X_train
        Y_train = self.Y_train
        prediction = kernel_extrapolate(X_train, test, Y_train, self.parameters, nsteps,self.kernel_keyword, regu_lambda = regu_lambda) 

        return prediction

def fit_data_anl3(train_data,delay,regu_lambda,noptsteps=100):
    lenX=len(train_data[0,:])
    num_modes = train_data.shape[0]

    # Some constants
    nparameters=24
    vdelay=delay*np.ones((num_modes,), dtype=int)
    vregu_lambda=regu_lambda*np.ones((num_modes,))

    # Get scaling factor    
    normalize=np.amax(train_data[:,:])

    # Prepare training data
    X=np.zeros((1+lenX-2*delay,delay*num_modes))
    Y=np.zeros((1+lenX-2*delay,delay*num_modes))
    for mode in range(train_data.shape[0]):
        for i in range(1+lenX-2*delay):
              X[i,(mode*delay):(mode*delay+delay)]=train_data[mode,i:(i+delay)]
              Y[i,(mode*delay):(mode*delay+delay)]=train_data[mode,(i+delay):(i+2*delay)]

    # Normalize
    X=X/normalize
    Y=Y/normalize     
    
    # Fit data
    c=np.zeros(nparameters)+1
    mu_1 = c
    kerneltype="RBF"
    K = KernelFlowsP(kerneltype, mu_1)
    mu_pred = K.fit(X, Y, noptsteps, optimizer = "SGD",  batch_size = 100, show_it = 500, regu_lambda=regu_lambda)
    mu_1=mu_pred
    c=mu_1

    kernel = kernels_dic[kerneltype]
    k_matrix = kernel(X, X, mu_1)
    k_matrix += regu_lambda * np.identity(k_matrix.shape[0])
    A=np.matmul(np.linalg.inv(k_matrix), Y)

    return k_matrix, A, mu_1, normalize, X

def fit_error_model_anl3(train_data,error_data,delay,regu_lambda,noptsteps=100):
    lenX=len(train_data[0,:])
    num_modes = train_data.shape[0]

    # Some constants
    nparameters=24
    vdelay=delay*np.ones((num_modes,), dtype=int)
    vregu_lambda=regu_lambda*np.ones((num_modes,))

    # Get scaling factor    
    normalize=np.amax(train_data[:,:])

    # Prepare training data
    X=np.zeros((1+lenX-2*delay,delay*num_modes))
    Y=np.zeros((1+lenX-2*delay,delay*num_modes))
    for mode in range(train_data.shape[0]):
        for i in range(1+lenX-2*delay):
              X[i,(mode*delay):(mode*delay+delay)]=train_data[mode,i:(i+delay)]
              Y[i,(mode*delay):(mode*delay+delay)]=error_data[mode,(i+delay):(i+2*delay)]

    # Normalize
    X=X/normalize
    Y=Y/normalize     
    
    # Fit data
    c=np.zeros(nparameters)+1
    mu_1 = c
    kerneltype="RBF"
    K = KernelFlowsP(kerneltype, mu_1)
    mu_pred = K.fit(X, Y, noptsteps, optimizer = "Nesterov",  batch_size = 100, show_it = 500, regu_lambda=regu_lambda)
    mu_1=mu_pred
    c=mu_1

    kernel = kernels_dic[kerneltype]
    k_matrix = kernel(X, X, mu_1)
    k_matrix += regu_lambda * np.identity(k_matrix.shape[0])
    A=np.matmul(np.linalg.inv(k_matrix), Y)

    return k_matrix, A, mu_1, normalize, X

def test_fit_anl3(test_data,train_X,delay,k_matrix,A,param,normalize):
    lenXt=len(test_data[0,:])
    num_modes = test_data.shape[0]

    nsteps=lenXt
    X_test=np.zeros((1,delay*num_modes))
    X_test[0,:] = test_data[:,:delay].reshape(1,-1)
    X_test=X_test/normalize
    
    arr = np.zeros((nsteps,num_modes))
    arr[:2*delay,:] = train_data[:,:2*delay].T/normalize

    kerneltype = "RBF"
    kernel = kernels_dic[kerneltype]

    X_test0=X_test
    isteps=int(nsteps/delay)+1
    for i in range(1,isteps):
        X_test1=X_test0
        t_matrix = kernel(X_test1, train_X, param) 
        prediction = np.matmul(t_matrix, A) 
        
        # Just using true data
        X_test0 = (test_data[:,i*delay:(i+1)*delay]/normalize).reshape(1,-1)

        # # Feedback
        # X_test0=prediction
        
        for mode in range(num_modes):
            imin=i*delay
            imax=np.minimum((i+1)*delay,nsteps)
            delaymin=imax-imin
            arr[imin:imax,mode]=np.array(prediction[0,(mode*delay):(mode*delay+delaymin)])

    # Rescale
    predall=arr*normalize

    return predall.T

def test_error_model_anl3(test_data,train_X,delay,k_matrix,A,param,normalize):
    lenXt=len(test_data[0,:])
    num_modes = test_data.shape[0]

    nsteps=lenXt
    X_test=np.zeros((1,delay*num_modes))
    X_test[0,:] = test_data[:,:delay].reshape(1,-1)
    X_test=X_test/normalize
    
    arr = np.zeros((nsteps,num_modes))
    arr[:2*delay,:] = train_data[:,:2*delay].T/normalize

    kerneltype = "RBF"
    kernel = kernels_dic[kerneltype]

    X_test0=X_test
    isteps=int(nsteps/delay)+1
    for i in range(1,isteps):
        X_test1=X_test0
        t_matrix = kernel(X_test1, train_X, param) 
        prediction = np.matmul(t_matrix, A) 
        
        # Just using true data
        X_test0 = (test_data[:,i*delay:(i+1)*delay]/normalize).reshape(1,-1)

        # # Feedback
        # X_test0=prediction
        
        for mode in range(num_modes):
            imin=i*delay
            imax=np.minimum((i+1)*delay,nsteps)
            delaymin=imax-imin
            arr[imin:imax,mode]=np.array(prediction[0,(mode*delay):(mode*delay+delaymin)])

    # Rescale
    predall=arr*normalize

    return predall.T



if __name__ == '__main__':
    #train_data = np.load('POD_Coeffs_train.npy')
    #test_data = np.load('POD_Coeffs_test.npy')

    # Set delay
    delay = 5
    regu_lambda = 0.010
    #noptsteps = 1000
    noptsteps = len(train_times)

    # Fit on training data
    k_matrix, A, param, normalize, train_X = fit_data_anl3(train_data,delay,regu_lambda,noptsteps)

    # Predict and get error on training data
    predicted_train = test_fit_anl3(train_data, train_X, delay, k_matrix, A, param, normalize)


    #np.save('RKHS_Train_Prediction.npy',predicted_train)
    #train_error = np.abs(predicted_train-train_data)
    #np.save('RKHS_Train_Error.npy',train_error)
    test_data= Data[:600,:].T

    # Predict testing data
    predicted_test = test_fit_anl3(test_data, train_X, delay, k_matrix, A, param, normalize)
    #np.save('RKHS_Test_Prediction.npy',predicted_test)
    test_error = np.abs(predicted_test-test_data)
   # np.save('RKHS_Test_Error.npy',train_error)

    # Visualize the modal predictions
    for mode_num in range(2):
        fig, ax = plt.subplots(nrows=1,ncols=2)
        ax[0].plot(predicted_test[mode_num,:],label='Test predicted')
        ax[0].plot(test_data[mode_num,:],label='Test true')
        ax[0].legend()

        ax[1].plot(predicted_train[mode_num,:],label='Train predicted')
        ax[1].plot(train_data[mode_num,:],label='Train true')
        ax[1].legend()
        plt.tight_layout()
        plt.show()

    #exit()

    # Fit an error model
    #k_matrix_err, A_err, param_err, normalize_err, train_X_err = fit_error_model_anl3(train_data,train_error,delay,regu_lambda,noptsteps)

    # Predict the error on train data
    #predicted_train_error = test_error_model_anl3(train_data, train_X_err, 
    #                                    delay, k_matrix_err, A_err, param_err, normalize_err)

    # Predict the error on test data
    #predicted_test_error = test_error_model_anl3(test_data, train_X_err, 
    #                                    delay, k_matrix_err, A_err, param_err, normalize_err)